In [12]:
import queue 

q = queue.Queue()

q.put(1)
q.put(2)
q.put(3)

print(q.empty())
print(q.get())
print(q.get())
print(q.get())
print(q.empty())

False
1
2
3
True


In [25]:
q = queue.Queue(maxsize=10)

for i in range(5):
    q.put(i,block=False)
    print(f"put {i}")

    
while True:
# while q.qsize()>0:
    print(q.get(block=False))

put 0
put 1
put 2
put 3
put 4
0
1
2
3
4


Empty: 

In [1]:
### ref: https://shengyu7697.github.io/python-queue/
### Producer / Consumer with Queue !!!
import queue , time ,random , threading

q = queue.Queue(10)

class Producer(threading.Thread):
    def __init__(self,thread_name):
        super(Producer,self).__init__(name=thread_name)
        
    def run(self):
        global q 
        count = 1
        while True:
            if q.full():
                print("queue is full")
                pass
            
            msg = str(count)
            q.put(msg)
            print(f'{self.name} put {msg} , qsize : {str(q.qsize())}\n')
            
            count = count +1
            time.sleep(random.random()*2)
        
class Consumer(threading.Thread):
    def __init__(self,thread_name):
        super(Consumer,self).__init__(name=thread_name)
        
    def run(self):
        global q
        while True:
            if q.empty():
                print("queue is empty")
                pass
            
            msg = q.get()
            print(f'{self.name} get {msg} , qsize : {str(q.qsize())}\n')
            time.sleep(random.random()*3)
        
        
p = Producer("producer")
p.start()

c = Consumer("consumer")
c.start()


In [2]:
# ref: https://blog.csdn.net/brucewong0516/article/details/84587522
# Producer / Consumer with Condition !!!
import threading , time

class Producer(threading.Thread):
    def run(self):
        global cnt
        while True:
            ### 此為 critical section 啦, 有 mutex 的人才可執行！
            ### con.acquire() 取得某一個有 mutex 的 thread , 其餘只能繼續 waiting 
            if con.acquire():
                if cnt >900:
                    con.wait()
                else:
                    cnt += 100
                    msg = f"{self.name} produce 100, count ={str(cnt)}"
                    print(msg)
                    
                    # 從 waiting 池挑一個 thread 出來
                    # 通知其用 acquire 方法, 嘗試取得 mutex
                    con.notify() 
                    
                con.release()
                time.sleep(1)
            ###
                
class Consumer(threading.Thread):
    def run(self):
        global cnt
        while True:
            if con.acquire():
                if cnt < 100:
                    con.wait()
                else:
                    cnt -= 30
                    msg = f"{self.name} consume 30 , count = {cnt}"
                    print(msg)
                    
                    con.notify()
                con.release()
                time.sleep(1)
                
                

cnt = 500
con = threading.Condition()


def main():
    for i in range(3):
        p = Producer()
        p.start()
        
    for j in range(20):
        c = Consumer()
        c.start()
        
# main()

In [50]:
### ref: https://www.pythontutorial.net/python-concurrency/python-threading-lock/
### Race condition with multi-thread
### And using lock (mutex) for critical section 
import threading , time 

counter = 0

mutex = threading.Lock()

def increase(by,lock):
    global counter 
    
    
    ### 沒有 lock(mutex), counter 值會亂存取 -> race condition 大炸裂
#     local_count = counter 
#     local_count += by 

#     time.sleep(0.1)

#     counter = local_count
#     print(f"counter={counter}")
    
    
    ### 使用 with 語法, 建構 critical section 
    with lock:
        local_count = counter 
        local_count += by 

        time.sleep(0.1)  ## 造成 race condition 的元兇！？ （發生 thread-level 的 context-switch ）

        counter = local_count
        print(f"counter={counter}")
    
    
    ### 使用 lock.acquire() / lock.release() 打造 critical section !!!
    ### 有 lock (mutex) 者, 才可進入此區執行
#     lock.acquire()
#     local_count = counter 
#     local_count += by 
    
#     time.sleep(0.1)
    
#     counter = local_count
#     print(f"counter={counter}")
#     lock.release()
    

t1 = threading.Thread(target=increase,args=(10,mutex))
t2 = threading.Thread(target=increase,args=(20,mutex))
t3 = threading.Thread(target=increase,args=(30,mutex))

t1.start()
t2.start()
t3.start()


t1.join()
t2.join()
t3.join()

print(f"The final counter is {counter}")

counter=10
counter=30
counter=60
The final counter is 60


In [46]:
### 操作 list 引發 race condition 失敗～～～

# testqq = []

# for i in range(1000):
#     testqq.append(1)
# print(len(testqq))

import threading ,time

testqq = []

def add_element(cnt):
    global testqq 
    
    local_list = testqq
    for i in range(cnt):
        local_list.append(1)
    
    time.sleep(0.1)
    
    testqq = local_list

        
t1 = threading.Thread(target=add_element,args=(300,))
t2 = threading.Thread(target=add_element,args=(180,))

t1.start()
t2.start()

t1.join()
t2.join()


print(f"The final list length: {len(testqq)}")
# print(f"The final list length: {str(total)}")

The final list length: 480
